In [8]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
import matplotlib.pyplot as plt
from scipy.stats import multivariate_normal
import tensorflow as tf
from tensorflow.keras import layers

# Load the data
data = {
    'Forces': ['10 N', '15 N', '20 N', '25 N', '35 N', '30 N', '40 N', '45 N', '50 N', '55 N', '60 N', '65 N', '70 N', '75 N'],
    'rms 1': [0.09, 0.22, 0.1, 0.19, 0.09, 0.11, 0.1, 0.08, 0.08, 0.09, 0.08, 0.07, 0.06, 0.05],
    'rms 2': [0.25, 0.29, 0.34, 0.2, 0.2, 0.14, 0.22, 0.12, 0.14, 0.09, 0.09, 0.06, 0.12, 0.1],
    'rms 3': [0.2, 0.14, 0.07, 0.11, 0.22, 0.13, 0.12, 0.11, 0.13, 0.09, 0.07, 0.1, 0.09, 0.05]
}

df = pd.DataFrame(data)

# Split the data into input (X) and output (y) variables
X = df[['Forces']]
y = df[['rms 1', 'rms 2', 'rms 3']]

# Convert the 'Forces' column to numeric values
X['Forces'] = X['Forces'].str.replace(' N', '').astype(int)


# Scale the input variables
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X)

# Reshape input to 3D tensor [samples, time steps, features]
X_reshaped = X_scaled.reshape(X_scaled.shape[0], 1, X_scaled.shape[1])

# Define the armor model
def build_armor_model():
    model = tf.keras.Sequential()

    # Add LSTM layer
    model.add(layers.LSTM(64, activation='relu', input_shape=(1, 1)))

    # Add fully connected layers
    model.add(layers.Dense(64, activation='relu'))
    model.add(layers.Dense(3))

    return model

# Create an instance of the armor model
armor_model = build_armor_model()

# Compile the model
armor_model.compile(optimizer='adam',
                    loss=tf.keras.losses.MeanSquaredError(),
                    metrics=['mse'])

# Train the model
armor_model.fit(X_reshaped, y, epochs=200, batch_size=10, verbose=1)


# Function to predict rms based on force input
def predict_rms(force):
    force_scaled = scaler.transform([[force]])
    prediction = armor_model.predict(force_scaled.reshape(1, 1, 1))
    return prediction

# Function to get the accurate rms value
def get_accurate_rms(predicted_rms):
    mean_rms = np.mean(predicted_rms, axis=0)
    return mean_rms


# Load the data
data = {
    'Forces': ['10 N', '15 N', '20 N', '25 N', '35 N', '30 N', '40 N', '45 N', '50 N', '55 N', '60 N', '65 N', '70 N', '75 N'],
    'HOFMs 1': [0.07, 0.21, 0.08, 0.17, 0.06, 0.09, 0.07, 0.06, 0.07, 0.07, 0.06, 0.04, 0.04, 0.02],
    'HOFMs 2': [0.25, 0.29, 0.34, 0.2, 0.2, 0.14, 0.22, 0.12, 0.14, 0.09, 0.09, 0.06, 0.12, 0.1],
    'HOFMs 3': [0.2, 0.14, 0.07, 0.11, 0.22, 0.13, 0.12, 0.11, 0.13, 0.09, 0.07, 0.1, 0.09, 0.05]
}

df = pd.DataFrame(data)

# Split the data into input (X) and output (y) variables
X = df[['Forces']]
y = df[['HOFMs 1', 'HOFMs 2', 'HOFMs 3']]

# Convert the 'Forces' column to numeric values
X['Forces'] = X['Forces'].str.replace(' N', '').astype(int)


# Function to predict HOFMs based on force input
def predict_hofms(force):
    force_scaled = scaler.transform([[force]])
    prediction = armor_model.predict(force_scaled.reshape(1, 1, 1))
    return prediction

# Function to get the accurate HOFMs value
def get_accurate_hofms(predicted_hofms):
    mean_hofms = np.mean(predicted_hofms, axis=0)
    return mean_hofms


num_samples = 999  # Number of samples in the generated EMG signal
num_channels = 1  # Number of channels in the EMG signal
num_clusters = 3  # Number of clusters in the Gaussian Mixture Model

def generate_zero_mean_gmm(num_clusters, num_samples):
    # Generate covariance matrices for each cluster
    covariances = [np.eye(num_samples) for _ in range(num_clusters)]  # Identity matrix as covariance for each cluster

    # Generate weights for each cluster (uniform weights)
    weights = np.ones(num_clusters) / num_clusters

    # Generate GMM samples
    gmm_samples = np.concatenate([
        multivariate_normal.rvs(mean=np.zeros(num_samples), cov=covariances[i], size=int(num_samples * weights[i]))
        for i in range(num_clusters)
    ])

    return gmm_samples

# Get force input from the user
force_input = 10

# Predict rms
rms_prediction = predict_rms(force_input)

# Predict HOFMs
hofms_prediction = predict_hofms(force_input)

# Get the accurate rms value
accurate_rms = get_accurate_rms(rms_prediction)

print('Predicted rms:', rms_prediction)
print('Accurate rms:', accurate_rms)

# Get the accurate HOFMs value
accurate_hofms = get_accurate_hofms(hofms_prediction)

print('Predicted HOFMs:', hofms_prediction)
print('Accurate HOFMs:', accurate_hofms)

# Generate GMM samples
gmm_samples = generate_zero_mean_gmm(num_clusters, num_samples)

# Example: Generate random 2nd order HOFMs values
hofms_value = np.array([accurate_hofms[0], accurate_hofms[1], accurate_hofms[2]])

# Example: Generate random RMS values
rms_value = np.array([accurate_rms[0], accurate_rms[1], accurate_rms[2]])

# Reshape hofms_value and rms_value to match the dimensions of gmm_samples
hofms_value = np.reshape(hofms_value, (-1, 1))
rms_value = np.reshape(rms_value, (1, -1))

# Combine HOFMs, RMS, and GMM
emg_signal = np.dot(hofms_value, rms_value) + gmm_samples

time = np.arange(num_samples)
plt.plot(time, emg_signal)
plt.xlabel('Time')
plt.ylabel('Amplitude')
plt.title('Generated EMG Signal')
plt.show()
print(emg_signal)


<ipython-input-8-4adc0db0745e>:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X['Forces'] = X['Forces'].str.replace(' N', '').astype(int)


Epoch 1/200
2/2 [==============================] - 2s 11ms/step - loss: 0.0212 - mse: 0.0212
Epoch 2/200
2/2 [==============================] - 0s 8ms/step - loss: 0.0185 - mse: 0.0185
Epoch 3/200
2/2 [==============================] - 0s 12ms/step - loss: 0.0165 - mse: 0.0165
Epoch 4/200
2/2 [==============================] - 0s 9ms/step - loss: 0.0149 - mse: 0.0149
Epoch 5/200
2/2 [==============================] - 0s 13ms/step - loss: 0.0135 - mse: 0.0135
Epoch 6/200
2/2 [==============================] - 0s 10ms/step - loss: 0.0122 - mse: 0.0122
Epoch 7/200
2/2 [==============================] - 0s 8ms/step - loss: 0.0112 - mse: 0.0112
Epoch 8/200
2/2 [==============================] - 0s 11ms/step - loss: 0.0102 - mse: 0.0102
Epoch 9/200
2/2 [==============================] - 0s 11ms/step - loss: 0.0093 - mse: 0.0093
Epoch 10/200
2/2 [==============================] - 0s 9ms/step - loss: 0.0086 - mse: 0.0086
Epoch 11/200
2/2 [==============================] - 0s 13ms/step - loss: 

<ipython-input-8-4adc0db0745e>:88: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X['Forces'] = X['Forces'].str.replace(' N', '').astype(int)
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(


1/1 [==============================] - 0s 23ms/step


/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(


Predicted rms: [[0.1490933  0.27601728 0.15510324]]
Accurate rms: [0.1490933  0.27601728 0.15510324]
Predicted HOFMs: [[0.1490933  0.27601728 0.15510324]]
Accurate HOFMs: [0.1490933  0.27601728 0.15510324]


ValueError: ignored

In [ ]:
import numpy as np

# Input array
input_array = emg_signal
# Multiply by 100 and add 400
output_array = input_array * 100 + 400

print(output_array.flatten())

In [ ]:
import serial
import time

arduino=serial.Serial('COM1', 9600)
time.sleep(2)

print("Enter  1 to turn ON LED and 0 to turn OFF LED")

while 1:

    force_input = float(input('Enter the force value (in N): '))

    if force_input == 0:
        arduino.write(b'0')
        print("LED  turned ON")
    elif
        rms_prediction = predict_rms(force_input)
        hofms_prediction = predict_hofms(force_input)
        accurate_rms = get_accurate_rms(rms_prediction)
        accurate_hofms = get_accurate_hofms(hofms_prediction)
        gmm_samples = generate_zero_mean_gmm(num_clusters, num_samples)  # Generate GMM samples
        rms_value = accurate_hofms# Example: Generate random RMS values
        hofms_value = accurate_rms # Example: Generate random 2nd order HOFMs values
        emg_signal = hofms_value * rms_value + gmm_samples
        time = np.arange(num_samples)

The suitability of the ARMOR model versus the Artificial Neural Network (ANN) model for predicting sensitive values such as EMG signals in real time with high accuracy depends on several factors. Let's consider the characteristics and considerations for each model:

ARMOR Model:
- Advantages:
  - ARMOR models are specifically designed for time series forecasting and can capture the temporal dependencies in the data.
  - They can incorporate exogenous variables, which can be beneficial for capturing external influences on the EMG signal.
  - ARMOR models have a solid theoretical foundation and statistical interpretation.
  - They can provide interpretable coefficients that describe the relationships between the variables.
- Limitations:
  - ARMOR models assume linear relationships between variables, which may not be suitable for complex or nonlinear relationships in the EMG signal.
  - They require careful selection of model orders (p, q, r), which can be challenging and time-consuming.
  - ARMOR models may struggle to capture nonlinear or complex patterns in the EMG signal.

ANN Model:
- Advantages:
  - ANN models have the ability to capture complex and nonlinear relationships, making them more flexible in modeling the EMG signal.
  - They can automatically learn features and patterns from the data without the need for manual feature engineering.
  - ANN models can handle a large number of input variables and are capable of capturing high-dimensional relationships.
- Limitations:
  - ANN models are generally considered as black-box models, making it challenging to interpret the relationships learned by the model.
  - They may require a large amount of training data to perform well, especially for complex tasks like EMG signal prediction.
  - ANN models can be computationally expensive to train, which may be a concern for real-time applications.

Considering the above points, both the ARMOR model and ANN model have their strengths and limitations. In practice, the choice between the two models for predicting sensitive values such as EMG signals in real time with high accuracy depends on factors such as the specific characteristics of the data, the complexity of the relationships involved, the availability of training data, and the computational resources available.

It is recommended to conduct experiments and comparative analysis using both models on your specific dataset to determine which model performs better in terms of accuracy, real-time performance, interpretability, and other relevant criteria. Additionally, it is worth exploring other advanced techniques specifically designed for time series analysis, such as recurrent neural networks (RNNs) or Long Short-Term Memory (LSTM) networks, which may provide improved performance for time-dependent data like EMG signals.